# Black-Scholes Option Pricing with HPXPy

This notebook demonstrates parallel option pricing using the Black-Scholes model.

**Topics covered:**
- Black-Scholes formula implementation
- Broadcasting for batch pricing
- Greeks computation (Delta, Gamma, Vega)
- Performance comparison with NumPy

## 1. Setup

In [1]:
import time
import numpy as np
from scipy.stats import norm
import hpxpy as hpx

# Initialize HPX runtime
hpx.init()
print(f"HPXPy initialized with {hpx.num_threads()} threads")

HPXPy initialized with 12 threads


## 2. Black-Scholes Model

The Black-Scholes formula prices European call and put options:

$$C = S \cdot N(d_1) - K \cdot e^{-rT} \cdot N(d_2)$$

$$P = K \cdot e^{-rT} \cdot N(-d_2) - S \cdot N(-d_1)$$

Where:
- $d_1 = \frac{\ln(S/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$
- $d_2 = d_1 - \sigma\sqrt{T}$
- $S$ = stock price
- $K$ = strike price
- $r$ = risk-free rate
- $T$ = time to expiration
- $\sigma$ = volatility
- $N(x)$ = standard normal CDF

In [2]:
def black_scholes_numpy(S, K, T, r, sigma, option_type='call'):
    """
    Black-Scholes option pricing using NumPy.
    
    Parameters:
    -----------
    S : array-like - Current stock price
    K : array-like - Strike price
    T : array-like - Time to expiration (years)
    r : float - Risk-free interest rate
    sigma : array-like - Volatility
    option_type : str - 'call' or 'put'
    
    Returns:
    --------
    Option price(s)
    """
    S = np.asarray(S, dtype=np.float64)
    K = np.asarray(K, dtype=np.float64)
    T = np.asarray(T, dtype=np.float64)
    sigma = np.asarray(sigma, dtype=np.float64)
    
    sqrt_T = np.sqrt(T)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt_T)
    d2 = d1 - sigma * sqrt_T
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price

# Test with single option
S, K, T, r, sigma = 100, 100, 1.0, 0.05, 0.2
call_price = black_scholes_numpy(S, K, T, r, sigma, 'call')
put_price = black_scholes_numpy(S, K, T, r, sigma, 'put')

print(f"Stock: ${S}, Strike: ${K}, T: {T}yr, r: {r*100}%, vol: {sigma*100}%")
print(f"Call price: ${call_price:.4f}")
print(f"Put price:  ${put_price:.4f}")
print(f"Put-Call Parity check: {call_price - put_price:.4f} vs {S - K*np.exp(-r*T):.4f}")

Stock: $100, Strike: $100, T: 1.0yr, r: 5.0%, vol: 20.0%
Call price: $10.4506
Put price:  $5.5735
Put-Call Parity check: 4.8771 vs 4.8771


## 3. HPXPy Implementation

We implement Black-Scholes using HPXPy's parallel operations.

In [3]:
def black_scholes_hpxpy(S, K, T, r, sigma, option_type='call'):
    """
    Black-Scholes option pricing using HPXPy.
    
    Uses native broadcasting for batch pricing - scalars and arrays can be mixed.
    """
    # Convert inputs to HPXPy arrays (handles scalars and arrays uniformly)
    S_arr = hpx.array(np.asarray(S, dtype=np.float64))
    K_arr = hpx.array(np.asarray(K, dtype=np.float64))
    T_arr = hpx.array(np.asarray(T, dtype=np.float64))
    sigma_arr = hpx.array(np.asarray(sigma, dtype=np.float64))
    
    # Compute d1 and d2 - broadcasting handles scalar/array combinations
    sqrt_T = hpx.sqrt(T_arr)
    sigma_sqrt_T = sigma_arr * sqrt_T
    
    d1 = (hpx.log(S_arr / K_arr) + (r + 0.5 * sigma_arr * sigma_arr) * T_arr) / sigma_sqrt_T
    d2 = d1 - sigma_sqrt_T
    
    # Convert to numpy for CDF (HPXPy doesn't have erf yet)
    d1_np = d1.to_numpy()
    d2_np = d2.to_numpy()
    
    if option_type == 'call':
        # C = S*N(d1) - K*exp(-rT)*N(d2)
        nd1 = hpx.array(norm.cdf(d1_np))
        nd2 = hpx.array(norm.cdf(d2_np))
        price = S_arr * nd1 - K_arr * hpx.exp(-r * T_arr) * nd2
    else:
        # P = K*exp(-rT)*N(-d2) - S*N(-d1)
        nmd1 = hpx.array(norm.cdf(-d1_np))
        nmd2 = hpx.array(norm.cdf(-d2_np))
        price = K_arr * hpx.exp(-r * T_arr) * nmd2 - S_arr * nmd1
    
    return price

# Test HPXPy implementation
call_hpx = black_scholes_hpxpy(S, K, T, r, sigma, 'call')
result = call_hpx.to_numpy()
# Handle both 0-d (scalar input) and 1-d (array input) results
price_value = float(result) if result.ndim == 0 else result[0]
print(f"HPXPy Call price: ${price_value:.4f}")

HPXPy Call price: $10.4506


## 4. Batch Option Pricing

Price many options at once using broadcasting.

In [4]:
# Price options across a range of strikes
S = 100.0
strikes = np.linspace(80, 120, 21)
T = 0.5
r = 0.05
sigma = 0.25

# NumPy batch pricing
call_prices_np = black_scholes_numpy(S, strikes, T, r, sigma, 'call')

# HPXPy batch pricing
call_prices_hpx = black_scholes_hpxpy(S, strikes, T, r, sigma, 'call')

print("Option Chain (S=$100, T=0.5yr, vol=25%)")
print(f"{'Strike':>8} {'Call (NumPy)':>14} {'Call (HPXPy)':>14}")
print("-" * 40)

for i, K in enumerate(strikes[::2]):  # Every other strike
    idx = i * 2
    print(f"${K:>7.0f} ${call_prices_np[idx]:>13.4f} ${call_prices_hpx.to_numpy()[idx]:>13.4f}")

Option Chain (S=$100, T=0.5yr, vol=25%)
  Strike   Call (NumPy)   Call (HPXPy)
----------------------------------------
$     80 $      22.5415 $      22.5415
$     84 $      19.1104 $      19.1104
$     88 $      15.9230 $      15.9230
$     92 $      13.0304 $      13.0304
$     96 $      10.4699 $      10.4699
$    100 $       8.2600 $       8.2600
$    104 $       6.4004 $       6.4004
$    108 $       4.8735 $       4.8735
$    112 $       3.6491 $       3.6491
$    116 $       2.6890 $       2.6890
$    120 $       1.9517 $       1.9517


## 5. Volatility Surface

Compute option prices across strike and volatility dimensions using broadcasting.

In [5]:
# Create a grid of strikes and volatilities
strikes = np.linspace(80, 120, 41)  # 41 strikes
vols = np.linspace(0.1, 0.5, 21)    # 21 volatilities

# Use broadcasting: reshape for outer product
strikes_2d = strikes.reshape(-1, 1)   # (41, 1)
vols_2d = vols.reshape(1, -1)         # (1, 21)

S = 100.0
T = 0.5
r = 0.05

# Compute prices across the surface
start = time.perf_counter()
prices_np = black_scholes_numpy(S, strikes_2d, T, r, vols_2d, 'call')
np_time = time.perf_counter() - start

print(f"Computed {prices_np.size:,} option prices")
print(f"Shape: {prices_np.shape} (strikes x volatilities)")
print(f"NumPy time: {np_time*1000:.2f} ms")

# Show sample of surface
print(f"\nSample prices (strike x vol):")
print(f"Strike\\Vol", end="")
for v in vols[::4]:
    print(f"{v*100:>8.0f}%", end="")
print()

for i in range(0, len(strikes), 8):
    print(f"${strikes[i]:>6.0f}", end="")
    for j in range(0, len(vols), 4):
        print(f"${prices_np[i, j]:>8.2f}", end="")
    print()

Computed 861 option prices
Shape: (41, 21) (strikes x volatilities)
NumPy time: 0.93 ms

Sample prices (strike x vol):
Strike\Vol      10%      18%      26%      34%      42%      50%
$    80$   21.98$   22.08$   22.64$   23.63$   24.92$   26.38
$    88$   14.21$   14.83$   16.10$   17.70$   19.46$   21.31
$    96$    7.02$    8.74$   10.72$   12.79$   14.89$   17.00
$   104$    2.18$    4.43$    6.68$    8.94$   11.19$   13.43
$   112$    0.37$    1.92$    3.91$    6.06$    8.27$   10.51
$   120$    0.03$    0.72$    2.16$    4.00$    6.03$    8.17


## 6. Option Greeks

Compute sensitivities (Greeks) using parallel operations.

**Delta**: $\Delta = \frac{\partial C}{\partial S} = N(d_1)$

**Gamma**: $\Gamma = \frac{\partial^2 C}{\partial S^2} = \frac{N'(d_1)}{S \sigma \sqrt{T}}$

**Vega**: $\nu = \frac{\partial C}{\partial \sigma} = S \sqrt{T} N'(d_1)$

**Theta**: $\Theta = \frac{\partial C}{\partial T}$

In [6]:
def compute_greeks_numpy(S, K, T, r, sigma):
    """Compute option Greeks."""
    S = np.asarray(S, dtype=np.float64)
    K = np.asarray(K, dtype=np.float64)
    
    sqrt_T = np.sqrt(T)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt_T)
    d2 = d1 - sigma * sqrt_T
    
    # N(d1) and N'(d1)
    Nd1 = norm.cdf(d1)
    Nd1_prime = norm.pdf(d1)  # Standard normal PDF
    
    delta = Nd1
    gamma = Nd1_prime / (S * sigma * sqrt_T)
    vega = S * sqrt_T * Nd1_prime / 100  # Per 1% vol change
    theta = -(S * Nd1_prime * sigma) / (2 * sqrt_T) - r * K * np.exp(-r * T) * norm.cdf(d2)
    theta = theta / 365  # Per day
    
    return {'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta}

# Compute Greeks for various strikes
S = 100.0
strikes = np.linspace(80, 120, 9)
T = 0.5
r = 0.05
sigma = 0.25

greeks = compute_greeks_numpy(S, strikes, T, r, sigma)

print("Option Greeks (S=$100, T=0.5yr, vol=25%)")
print(f"{'Strike':>8} {'Delta':>10} {'Gamma':>10} {'Vega':>10} {'Theta':>10}")
print("-" * 52)

for i, K in enumerate(strikes):
    print(f"${K:>7.0f} {greeks['delta'][i]:>10.4f} {greeks['gamma'][i]:>10.4f} "
          f"{greeks['vega'][i]:>10.4f} {greeks['theta'][i]:>10.4f}")

Option Greeks (S=$100, T=0.5yr, vol=25%)
  Strike      Delta      Gamma       Vega      Theta
----------------------------------------------------
$     80     0.9322     0.0074     0.0927    -0.0160
$     85     0.8748     0.0117     0.1458    -0.0195
$     90     0.7955     0.0160     0.2006    -0.0227
$     95     0.6985     0.0197     0.2464    -0.0249
$    100     0.5909     0.0220     0.2747    -0.0258
$    105     0.4816     0.0225     0.2818    -0.0251
$    110     0.3785     0.0215     0.2689    -0.0230
$    115     0.2875     0.0193     0.2410    -0.0200
$    120     0.2114     0.0164     0.2046    -0.0166


## 7. Performance Benchmark

In [7]:
def benchmark_pricing(n_options, warmup=3, repeats=10):
    """Benchmark option pricing."""
    # Generate random option parameters
    S = np.random.uniform(80, 120, n_options)
    K = np.random.uniform(80, 120, n_options)
    T = np.random.uniform(0.1, 2.0, n_options)
    sigma = np.random.uniform(0.1, 0.5, n_options)
    r = 0.05
    
    # Warmup
    for _ in range(warmup):
        black_scholes_numpy(S[:100], K[:100], T[:100], r, sigma[:100])
    
    # Time NumPy
    np_times = []
    for _ in range(repeats):
        start = time.perf_counter()
        prices_np = black_scholes_numpy(S, K, T, r, sigma)
        np_times.append(time.perf_counter() - start)
    
    return min(np_times) * 1000

print("Black-Scholes Pricing Performance")
print(f"{'Options':>12} {'NumPy (ms)':>12} {'Options/ms':>12}")
print("=" * 40)

for n in [10000, 100000, 1000000]:
    np_time = benchmark_pricing(n)
    throughput = n / np_time
    print(f"{n:>12,} {np_time:>12.2f} {throughput:>12,.0f}")

Black-Scholes Pricing Performance
     Options   NumPy (ms)   Options/ms


      10,000         0.52       19,237
     100,000         4.28       23,362


   1,000,000        46.79       21,374


## 8. Cleanup

In [8]:
hpx.finalize()
print("HPX runtime finalized")

HPX runtime finalized


## Summary

The Black-Scholes model is a great example of financial computation that benefits from:

1. **Vectorization** - Price thousands of options simultaneously
2. **Broadcasting** - Compute option surfaces efficiently
3. **Parallel Math** - HPXPy parallelizes exp, log, sqrt operations

### Extensions

- Add implied volatility solver (Newton-Raphson)
- Monte Carlo pricing for exotic options
- Binomial tree pricing
- Portfolio risk metrics (VaR, Expected Shortfall)

### Further Reading

- [Black-Scholes Model](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model)
- [Option Greeks](https://en.wikipedia.org/wiki/Greeks_(finance))
- [Monte Carlo Methods in Finance](https://en.wikipedia.org/wiki/Monte_Carlo_methods_in_finance)